# PCML Project-1 ~ Team #60

## Initial Python Imports

In [78]:
# Useful starting lines
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import datetime

from costs import compute_loss

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [79]:
from proj1_helpers import *
from helpers import *

DATA_TRAIN_PATH = "../Data/train.csv" # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

#print the shape of the offset x matrix.
print(tX.shape)

(250000, 30)


## Data preprocessing

### Standardization

In [80]:
#standardization
tX, mean_x, std_x = standardize(tX, mean_x=None, std_x=None)

## Linear regression using gradient descent

In [81]:
from gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.01

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start gradient descent.
# start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y, tX, w_initial, max_iters, gamma)
# end_time = datetime.datetime.now()

print('parameters w: ',gradient_ws[-1])


Gradient Descent(0/999): loss=0.5
Gradient Descent(1/999): loss=0.4930403211757793
Gradient Descent(2/999): loss=0.4870098501022072
Gradient Descent(3/999): loss=0.4817168268664753
Gradient Descent(4/999): loss=0.4770128047663221
Gradient Descent(5/999): loss=0.4727827104481938
Gradient Descent(6/999): loss=0.4689371916527971
Gradient Descent(7/999): loss=0.4654067258061777
Gradient Descent(8/999): loss=0.4621370840030196
Gradient Descent(9/999): loss=0.45908583830059513
Gradient Descent(10/999): loss=0.45621967211044767
Gradient Descent(11/999): loss=0.4535123087925472
Gradient Descent(12/999): loss=0.45094291613495796
Gradient Descent(13/999): loss=0.4484948771748208
Gradient Descent(14/999): loss=0.4461548430418665
Gradient Descent(15/999): loss=0.4439120029218537
Gradient Descent(16/999): loss=0.4417575211821269
Gradient Descent(17/999): loss=0.4396841032046329
Gradient Descent(18/999): loss=0.43768566032584444
Gradient Descent(19/999): loss=0.43575705109814233
Gradient Descent(20/

## Linear regression using stochastic gradient descent

In [82]:
from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.01
batch_size = 50

# Initialization
w_initial = np.zeros(tX.shape[1])

# Start SGD.
# start_time = datetime.datetime.now()
stoch_gradient_losses, stoch_gradient_ws = stochastic_gradient_descent(y, tX, w_initial, batch_size, max_iters, gamma)
# end_time = datetime.datetime.now()

print(stoch_gradient_ws[-1])

Stochastic Gradient Descent(0/999): loss=0.5
Stochastic Gradient Descent(1/999): loss=0.49222589205740186
Stochastic Gradient Descent(2/999): loss=0.49135157855815487
Stochastic Gradient Descent(3/999): loss=0.4843033379971171
Stochastic Gradient Descent(4/999): loss=0.47046065605389237
Stochastic Gradient Descent(5/999): loss=0.43846784661884025
Stochastic Gradient Descent(6/999): loss=0.45485477450700784
Stochastic Gradient Descent(7/999): loss=0.4423179007530215
Stochastic Gradient Descent(8/999): loss=0.4538152752224161
Stochastic Gradient Descent(9/999): loss=0.45994359028621384
Stochastic Gradient Descent(10/999): loss=0.44927270145487597
Stochastic Gradient Descent(11/999): loss=0.46335742222087317
Stochastic Gradient Descent(12/999): loss=0.47571407608318245
Stochastic Gradient Descent(13/999): loss=0.42783913708759846
Stochastic Gradient Descent(14/999): loss=0.500655125307837
Stochastic Gradient Descent(15/999): loss=0.4609521743340496
Stochastic Gradient Descent(16/999): los

## Least squares regression using normal equations

In [83]:
from least_squares import *

# start_ls_time = datetime.datetime.now()
ls_wopt, ls_loss = least_squares(y,tX)
print(ls_loss)
print(ls_wopt)
# end_ls_time = datetime.datetime.now()

0.339445598499
[ -3.14664000e-01   2.93790611e-02  -2.52531197e-01  -2.54790177e-01
  -3.03690297e-02  -1.40091635e+00   2.95716877e-01  -1.07873855e+01
   2.67880203e-01  -2.44944270e-03  -3.28831618e+02  -1.82647939e-01
   1.14039609e-01   2.05024908e+01   6.38861252e+01  -3.18964649e-04
  -1.80884217e-03   6.29952650e+01  -4.48641572e-04   1.54379298e-03
   1.21462700e-01   3.95268716e-04  -6.33223476e-02  -2.06747093e-01
  -1.16655766e-01   9.86256504e-02   1.67907699e-01  -3.35146241e-02
  -2.98358674e+00  -5.36388102e+00   2.78482381e+02]


## Ridge regression using normal equations

In [ ]:
from ridge_regression import ridge_regression

# lambdas = np.logspace(-3, 1, 10)      
# φ_x = build_poly(x, degree)
# x_train, x_test, y_train, y_test = split_data(tX, y, ratio, seed)
    
#     for lamb in lambdas:

w_ridge = ridge_regression(y, tX, 0.01)

print(w_ridge)

# rmse_tr = np.sqrt(2*compute_loss(y, tX, w_ridge))
# rmse_te = np.sqrt(2*compute_loss(y, tX, w_ridge))


[-0.30849412  0.03572402 -0.24077855 -0.21610016 -0.01054923 -0.01919922
  0.34814992 -0.03018983  0.23504977 -0.01013674 -0.00087247 -0.15674991
  0.11455458 -0.02161131  0.18503225 -0.00076269 -0.00130455  0.23850622
 -0.00086979  0.00249252  0.10383837  0.00113401 -0.06198121 -0.14991608
  0.03645409  0.04367213  0.0436772  -0.02328488 -0.02309077 -0.0234825
 -0.09703028]


## Logistic regression using gradient descent or SGD

## Regularized logistic regression using gradient descent or SGD

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = "../Data/test.csv" # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)